%%latex

Simulating a DDM means simulating the noisy trajectories that the Decision Variable (DV) follows from the starting point until they reach one of the thresholds, which determines the Reaction Time (RT, the time at which the threshold was hit) and the choice (which threshold was hit). Mathematically, the sub-threshold evolution of the decision variable $x$ is described by the following dynamics (in what's called the Langevin Equation) 

\begin{equation}
    dx = \nu dt + \sigma dB_t \quad x(t=0) = x_0
\end{equation}
The bounds are located at $\pm \theta$. The parameters $\nu$ and $\sigma^2$ measure, respectively, the mean and variance of the sensory evidence. What this equation is saying, intuitively, is that at every small increment of time $dt$, the decision variable increases by a constant term $\nu dt$, where $\nu$ is called the "drift" of the system (also often referred to as the strength of evidence), plus a stochastic term $\sigma dB_t$. The quantity $dBt$ can be understood as a normal (gaussian) random variable with mean zero and variance $dt$. This implies that its standard deviation is $\sqrt{dt}$, and so, because $dt$ is very small, it's going to much larger than the drift term. However, the drift term is consistent, while the noise term isn't (it has mean zero), so on average $x$ will evolve in the direction of the sign of $\nu$, and faster and more deterministicly the stronger $\nu$ is. 
Note that because the increment of $x$ does not depend on $x$, the `zero' value of the decision variable is arbitrary. That is why we can always choose the bounds as symmetric. 

It turns out that this system is overparametrized, meaning that you can find combinations of parameters that give you exactly the same distributions of RTs and choices. A common way to adress this is to remove $\sigma$ by normalizing the DV and the bounds relative to it:
\begin{equation}
    \sigma d\frac{x}{\sigma} = \sigma \frac{\nu}{\sigma} dt + \sigma dB_t 
\end{equation}
Thus, if instead of the original problem, we consider an alternative problem with the decision variable $x'=x/\sigma$ and $v \equiv \nu/\sigma$, we would obtain
\begin{equation}
    dx' = v dt + dB_t 
\end{equation}
with bounds at $x' = \pm \theta/\sigma$ and starting position $x'(t=0) = x_0/\sigma \equiv x'_0$. Since, as we said above, the origin of the decision variable is arbitrary, it is common to put the origin at the lower bound, so we define $\tilde{x} = x' - (-\theta/\sigma)$. Because this is just a shift in offset, the dynamical equation doesn't change
\begin{equation}
    d\tilde{x} = v dt + dB_t 
\end{equation}
the bounds for this problem are at $\tilde{x}=0$ and $2\theta/\sigma \equiv a$, and the initial condition is at $\tilde{x}(t=0) = x'_0+\theta/\sigma \equiv \tilde{x}_0$, which can vary between 0 and $a$. It is customary to normalize the initial condition in units of $a$, so that instead of $\tilde{x}_0$ one uses $w \equiv \tilde{x}_0/a$, which can take any value between 0 and 1. This version of the model, with parameters $\{v,w,a\}$, and is considered the "standard model" in the literature.



%%latex

That was some math! But don't worry, when you play with it you will see that it's not scary.
The first task will be to simulate this simple DDM. You just need to translate the math describing the evolution of the DV into code. 

Write a simple simulator function that takes as input the parameters $\{v,w,a\}$ and gives back $N$ choices and RT

In [1]:
import numpy as np

def simulator(v,w,a,N):

    # think of how to initialize things

    for k in range(N):
        # while bound has not been hit

        x[k,t_i] = x_[k,t_i-1] + ...

    choices[k] = ...
    RTs[k] = ...

    return choices, RTs, x

After you have the simulator, there's many things you can plot: RT distributions, psychometric functions, chronometric functions... I'll tell you more about it when you get there!

%%latex

Another parametrization, which we use in our Weber's Law paper, proceeds by defining a new decision variable $z$  equal to the original one $x$ in units of the decision bound, and a new time variable $\tau$ equal to the original one $t$ in units of the average time $t_{\theta}$ that it takes to hit the bound when the strength of evidence is zero (and which can be shown to be $t_{\theta} = (\theta/\sigma)^2$), so that
\begin{equation}
    z = x/\theta \quad \quad \tau = t/t_{\theta}
\end{equation}
Using these new variables, the Langevin equation reads
\begin{equation}
    dz = \mu d\tau + dB_{\tau} \quad z(t=0) = z_0 
\end{equation}
with $\mu = \theta\nu/\sigma^2$ and the decision bounds at $\pm 1$. This shows that the model in general has only two 'non-trivial' parameters: $\mu$ and $z_0$. The $z_0$ parameter, determines the bias of the psychometric function (i.e., the extent to which the probability of choosing either of the two options is different from 0.5 when the strength of evidence is zero). Because this feature of the data is very salient, one can typically know 'in advance' if $z_0$ is going to be needed. In our Weber's Law paper, there was very little bias, so we directly set $z_0=0$, in which case there is only 1 parameter for the decision process. In general, $\{ \mu, z_0\}$ determine the psychometric function and the shape of the Reaction Time distribution and how it depends on the strength of evidence. In addition, there is the parameter $t_{\theta}$, which depends on the variance of the evidence $\sigma^2$ then specifies the 'units of time' of the problem. So changing $\sigma^2$ while keeping $\nu/\sigma^2$ fixed has the effect of rigidly stretching the reaction time distributions without affecting its shape or the probability of hitting the bounds. 
%Instead, this probability depends only on the ratio $\theta \nu/\sigma^2$. 
The fact that this probability depends on the ratio between the mean and variance of the evidence is the reason why Weber's law holds whenever a particular change in the sensory stimulus has the effect of changing $\nu$ and $\sigma^2$ by the same amount (which was first noticed by Link 1992).


%% latex

So, going back to the first parametrization, we just understood above that from the 3 degrees of freedom, only two 
are important for the decision process, and the third only determines the units of time of the problem. There are two very similar ways of separating the units of time from the rest of the problem. One way is to change the scale of the decision variable so that the distance between the bounds is equal to 2. For this, we should use a new decision variable $z=\tilde{x}/(a/2)$ and a new temporal variable $\tau=t/(a/2)^2$, i.e., rescale time by $t_{\theta}=(a/2)^2$. The initial condition $w$ is already normalized between 0 and 1. Recalling that $dB_t = \sqrt{t_{\theta}}dB_{\tau}=adB_{\tau}$, the Langevin equation becomes
\begin{equation}
    dz = \mu d\tau +  dB_{\tau}  \quad z(\tau=0) = w % changed x to x'
\end{equation}
which depends only on two parameters: $\mu \equiv va/2$, and $w$.

This can be revealed explicitly by measuring the decision variable $\tilde{x}$ relative to $a$, i.e., $z=\tilde{x}/a$ and time $t$ relative to $t_{\theta}=a^2$, i.e., $\tau = t/a^2$. Recalling that $dB_t = \sqrt{t_{\theta}}dB_{\tau}=adB_{\tau}$, the previous equation becomes
\begin{equation}
    dz = \mu d\tau +  dB_{\tau}  \quad z(\tau=0) = w % changed x to x'
\end{equation}
which depends only on two parameters: $\mu \equiv va$, and $w$. 


The initial condition $w$ did not change because we were already measuring it relative to $a$. The parameter $a$ only determines the units of time. Note that this is identical to our description above, except that there the bounds were at $\pm 1$ and no bias meant $z_0 = 0$, and here the bounds are at 0 and 1 and no bias means $w=1/2$.

If we want to use the expressions for the normalized process used in our paper, they can be obtained by setting $w=1/2$, $a=2$ and $v=\mu$. Then, to return to the original units, we just need to rescale time by $t_{\theta} = (a/2)^2$.